In [179]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


I tried taking out other columns that I thought might be more noise than usefull, however nothing I removed resulted in better accuracy for the models

In [180]:
# We are going to see if dropping ASK_AMT & SPECIAL_CONSIDERATIONS helps our model:
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


I played around with different bucket sizes for the "other", and found the smaller the better the accuracy of the model

In [181]:
app_types = application_df['APPLICATION_TYPE'].value_counts()
# Determine which values to replace if counts are less than ...?
replace_application = list(app_types[app_types < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [182]:
class_types = application_df['CLASSIFICATION'].value_counts()
# Determine which values to replace if counts are less than ..?
replace_class = list(class_types[class_types < 350].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [183]:
# Generate our categorical variable lists
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [184]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [185]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [186]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns = "IS_SUCCESSFUL")
y = application_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [187]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Optimization with one extra hidden layer + increased nodes

In [188]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 10)                460       
_________________________________________________________________
dense_103 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_104 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_105 (Dense)            (None, 1)                 11        
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [189]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [190]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6012 - accuracy: 0.6975
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5624 - accuracy: 0.7258
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7340
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accura

Epoch 81/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5407 - accuracy: 0.7382
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5402 - accuracy: 0.7367
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5405 - accuracy: 0.7370
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5407 - accuracy: 0.7384
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5405 - accuracy: 0.7368
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5405 - accuracy: 0.7359
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5404 - accuracy: 0.7387
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5401 - accuracy: 0.7378
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5401 - accuracy: 0.7393
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5401

Accuracy = 73.76%

# Additional hidden layer +  additional 100 Epochs

In [191]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 10
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_106 (Dense)            (None, 10)                460       
_________________________________________________________________
dense_107 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_108 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_109 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_110 (Dense)            (None, 1)                 11        
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [193]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [194]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6005 - accuracy: 0.6926
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7285
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7304

Epoch 00003: saving model to checkpoints2\weights.03.hdf5
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7316
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7332
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7331
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7351
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7339

Epoch 00008: saving model to checkpoints2\weights.08.hdf5
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss:

804/804 [==============================] - 3s 3ms/step - loss: 0.5404 - accuracy: 0.7381

Epoch 00073: saving model to checkpoints2\weights.73.hdf5
Epoch 74/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5405 - accuracy: 0.7380
Epoch 75/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5398 - accuracy: 0.7377
Epoch 76/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5408 - accuracy: 0.7371
Epoch 77/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5400 - accuracy: 0.7381
Epoch 78/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5401 - accuracy: 0.7379

Epoch 00078: saving model to checkpoints2\weights.78.hdf5
Epoch 79/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5403 - accuracy: 0.7387
Epoch 80/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5401 - accuracy: 0.7380
Epoch 81/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5

804/804 [==============================] - 3s 3ms/step - loss: 0.5385 - accuracy: 0.7389
Epoch 145/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5386 - accuracy: 0.7393
Epoch 146/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5386 - accuracy: 0.7392
Epoch 147/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5384 - accuracy: 0.7388
Epoch 148/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5384 - accuracy: 0.7392

Epoch 00148: saving model to checkpoints2\weights.148.hdf5
Epoch 149/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5382 - accuracy: 0.7387
Epoch 150/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5382 - accuracy: 0.7394
Epoch 151/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5385 - accuracy: 0.7387
Epoch 152/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5385 - accuracy: 0.7386
Epoch 153/200
804/804 [====

Accuracy = 73.9%


# Another hidden layer, plus increased nodes, and 200 Epochs


In [195]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  25
hidden_nodes_layer2 = 25
hidden_nodes_layer3 = 25
hidden_nodes_layer4 = 25
hidden_nodes_layer5 = 25
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))
# Fifth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_111 (Dense)            (None, 25)                1150      
_________________________________________________________________
dense_112 (Dense)            (None, 25)                650       
_________________________________________________________________
dense_113 (Dense)            (None, 25)                650       
_________________________________________________________________
dense_114 (Dense)            (None, 25)                650       
_________________________________________________________________
dense_115 (Dense)            (None, 25)                650       
_________________________________________________________________
dense_116 (Dense)            (None, 1)                 26        
Total params: 3,776
Trainable params: 3,776
Non-trainable params: 0
___________________________________________________

In [196]:

os.makedirs("checkpoints3/",exist_ok=True)
checkpoint_path = "checkpoints3/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [ ]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5742 - accuracy: 0.7144
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7279
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7308
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7317
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7333
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7333
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7331
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7339
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7337
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5342 - accuracy: 0.7415
Epoch 82/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5343 - accuracy: 0.7409
Epoch 83/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5337 - accuracy: 0.7415
Epoch 84/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7412
Epoch 85/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7414
Epoch 86/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5337 - accuracy: 0.7408
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5337 - accuracy: 0.7407
Epoch 88/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5335 - accuracy: 0.7409
Epoch 89/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5330 - accuracy: 0.7406
Epoch 90/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5333 - accuracy: 

804/804 [==============================] - 2s 3ms/step - loss: 0.5311 - accuracy: 0.7417
Epoch 161/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5315 - accuracy: 0.7422
Epoch 162/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5304 - accuracy: 0.7425
Epoch 163/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5309 - accuracy: 0.7421
Epoch 164/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5306 - accuracy: 0.7425
Epoch 165/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5303 - accuracy: 0.7421
Epoch 166/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5304 - accuracy: 0.7421
Epoch 167/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5310 - accuracy: 0.7424
Epoch 168/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5312 - accuracy: 0.7418
Epoch 169/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5306 - a

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")